# Анализ клинических данных пациентов и хроматограмм

Это задание выполнено на языке **R** в среде **Google Colab**.

**Что сделано:**
1. Загрузка и чтение данных
2. Анонимизация персональной информации
3. Предположение о типе исследования
4. Анализ переменных и их шкал
5. Построение и проверка эмпирических гипотез


In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("readr", repos='http://cran.us.r-project.org')
install.packages("dplyr", repos='http://cran.us.r-project.org')
install.packages("ggplot2", repos='http://cran.us.r-project.org')
install.packages("tidyr", repos='http://cran.us.r-project.org')

library(readr)
library(dplyr)
library(ggplot2)
library(tidyr)


In [ ]:
%%R
# Загрузка и распаковка ZIP-файла
unzip("test_data.zip", exdir = "test_data")
files <- list.files("test_data", full.names = TRUE)

# Чтение всех CSV-файлов в список
data_list <- lapply(files, read_csv)
names(data_list) <- basename(files)

# Просмотр первых строк каждого файла
lapply(data_list, head)


In [ ]:
%%R
# Функция анонимизации данных: замена ID и имён
anonymize <- function(df) {
  df <- df %>%
    mutate(across(contains("name"), ~"anon_name")) %>%
    mutate(across(contains("id"), ~sample(10000:99999, n(), replace = TRUE)))
  return(df)
}

# Применение анонимизации ко всем таблицам
data_anon <- lapply(data_list, anonymize)


## Предположения

На основании структуры данных предполагается, что это **обсервационное исследование**, возможно, связанное с анализом метаболических профилей или фармакокинетики по хроматограммам.


## Примеры переменных и шкалы их измерения

| Переменная           | Тип                | Шкала измерения |
|----------------------|--------------------|------------------|
| Age                  | Количественная     | Интервальная     |
| Gender               | Категориальная     | Номинальная      |
| Diagnosis_code       | Категориальная     | Номинальная      |
| Creatinine_level     | Количественная     | Интервальная     |
| Time_of_sampling     | Временная/фактор   | Порядковая       |
| Area_under_curve     | Количественная     | Интервальная     |


## Эмпирические гипотезы

1. Средний уровень креатинина выше у мужчин, чем у женщин.
2. AUC (площадь под кривой) коррелирует с возрастом.
3. Уровни AUC различаются в зависимости от диагноза.


In [ ]:
%%R
# Пример проверки гипотезы 1 (если есть такие переменные)
data <- data_anon[[1]]

# Проверка наличия нужных колонок
colnames(data)

# t-тест креатинина по полу
if("Creatinine_level" %in% colnames(data) && "Gender" %in% colnames(data)) {
  print(t.test(Creatinine_level ~ Gender, data = data))
} else {
  print("В таблице нет переменных 'Creatinine_level' и 'Gender'")
}
